### Libraries, paths, and set-up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import datetime as dt
import os
os.chdir('..')
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from src.data.loader import data_loader
from src.vizs.imager import *
from src.utils.utils import *

if not os.path.exists('reports/images'):
    os.mkdir('reports/images')

We make use of a particularly defined function to load interim data

In [ ]:
# Processed dataframes
soi, capacity, askprice, bidprice, resource, demand, supply, price_fuel, mcost, generation = data_loader()

# Latest shares
share2022 = capacity[20933:]

# 2022-Gen-SIN
gen2022 = pd.read_excel('data/external/2022_GENERACIONSIN.xlsx',header=0, nrows=433, 
                        names=['plant','dispatch_type','gen_GWh','share %'])

### Histograms
At level and log transformations

In [ ]:
# Level Hist 1
data_list = [askprice, supply, bidprice, demand]
cols = ['daily_ask', 'supply_hourly', 'hourly_bid', 'demand_hourly']
names = ['Daily Ask Price', 'Hourly Supply', 'Hourly Bid Price', 'Hourly Demand']
x_titles = ['COP/kWh', 'kW', 'COP/kWh', 'kW']
colors = ['navy', 'blue', 'firebrick', 'red']
rows = [1, 1, 2, 2]
cols_nums = [1, 2, 1, 2]
ylogs = [False, False, False, False]
filename = 'images/histograms.pdf'
hist = create_subplots(data_list, cols, names, x_titles, colors, rows, cols_nums, ylogs, filename)

In [ ]:
# Level Hist 2
data_list = [capacity, resource, mcost, generation]
cols = ['capacity_kW', 'heat_rate', 'hourly_mc', 'generation_hourly']
names = ['Daily Installed Capacity', 'Heat Rate', 'Hourly Marginal Cost', 'Hourly Generation']
x_titles = ['kW', 'BTU', 'COP/kWh', 'kW']
colors = ['navy', 'blue', 'firebrick', 'red']
rows = [1, 1, 2, 2]
cols_nums = [1, 2, 1, 2]
ylogs = [False, False, False, False]
filename = 'images/histograms2.pdf'
hist = create_subplots(data_list, cols, names, x_titles, colors, rows, cols_nums, ylogs, filename)

In [ ]:
# Log Hist 1
data_list = [askprice, supply, bidprice, demand]
cols = ['daily_ask', 'supply_hourly', 'hourly_bid', 'demand_hourly']
names = ['Daily Ask Price', 'Hourly Supply', 'Hourly Bid Price', 'Hourly Demand']
x_titles = ['COP/kWh', 'kW', 'COP/kWh', 'kW']
colors = ['navy', 'blue', 'firebrick', 'red']
rows = [1, 1, 2, 2]
cols_nums = [1, 2, 1, 2]
ylogs = [True, True, True, True]
filename = 'images/loghistograms.pdf'
hist = create_subplots(data_list, cols, names, x_titles, colors, rows, cols_nums, ylogs, filename)

In [ ]:
# Level Hist 2
data_list = [capacity, resource, mcost, generation]
cols = ['capacity_kW', 'heat_rate', 'hourly_mc', 'generation_hourly']
names = ['Daily Installed Capacity', 'Heat Rate', 'Hourly Marginal Cost', 'Hourly Generation']
x_titles = ['kW', 'BTU', 'COP/kWh', 'kW']
colors = ['navy', 'blue', 'firebrick', 'red']
rows = [1, 1, 2, 2]
cols_nums = [1, 2, 1, 2]
ylogs = [True, True, True, True]
filename = 'images/loghistograms2.pdf'
hist = create_subplots(data_list, cols, names, x_titles, colors, rows, cols_nums, ylogs, filename)

### El Niño Phenomenon
Visualizing through our time period

In [ ]:
soi_month = soi.groupby('month')['soi'].mean().reset_index()
soi_month['color'] = np.where(soi_month['soi']<0, 'red', 'navy')

soi=px.bar(soi_month, x='month', y='soi',
       labels={'month':'Month', 'soi':'Southern Oscillation Index'}, template='plotly_white')
soi.update_traces(marker_color=soi_month["color"])
soi.update_layout(font_color='black',font_family='Latin Modern Math')

soi.show()
soi.write_image('images/SOI.pdf', scale=2)

### Market description
Visualizing capacity and plant participation

In [ ]:
# Shares x Technology @ 2022-12-31
values = share2022['capacity_kW'].unique().tolist()
labels = share2022['technology'].unique().tolist()
colors = ['darkgrey','navy', 'slategrey', 'red','dimgrey']
formatted_sum = "{:,.2f} GW".format(share2022['capacity_kW'].sum()/(10**6))

sharetech = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5,
                             marker=dict(colors=colors),
                             insidetextfont=dict(size=18),
                             outsidetextfont=dict(size=18))])

sharetech.update_layout(font_family='Latin Modern Math', template='plotly_white',
                 annotations=[dict(text=formatted_sum, 
                                   x=0.5, y=0.5, font_size=18, showarrow=False)])

sharetech.show()
sharetech.write_image('images/sharetech2022.pdf', scale=2)

In [ ]:
# Showcasing most relevant plants
gen2022top = gen2022[gen2022['share %'] > 0.01]

top20 = px.bar(gen2022top, x='plant', y='share %', labels={
                     'plant': 'Power Plant',
                     'share %': 'Market Share (%)'
                 }, 
             color_discrete_sequence=['navy'],
             template='plotly_white')

top20.update_yaxes(tickvals=[0, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12],  
                   ticktext=[0, 2, 4, 6, 8, 10, 12]) 

top20.add_annotation(x=19.5, y=0.1,
                     text=f'{gen2022["gen_GWh"].sum():.2f} GWh',
                     showarrow=False, font=dict(size=14))

top20.update_layout(xaxis_tickangle=-90)
top20.update_layout(font_color='black', font_family='Latin Modern Math')
top20.show()
top20.write_image('images/plants2022.pdf', scale=2)

### Prices and quantities
Visualizing market relevant information: demand, supply, ask (supply) price by fuel, bid (demand) price, and marginal cost

In [ ]:
# Supply DF
supply_fuel = supply.merge(resource[['plant','fuel','agent']], on='plant', how='left')
supply_fuel = datetimer(supply_fuel)
supply_fuel['date'] = supply_fuel['datetime'].dt.date
supply_fuel_summ = supply_fuel.groupby(['date', 'fuel'])['supply_hourly'].sum().reset_index()
supply_fuel_summ['supply_GWh'] = supply_fuel_summ['supply_hourly']/1000000
supply_fuel_summ = supply_fuel_summ.drop(columns=['supply_hourly'])

# Demand DF
demand_c = demand.copy()
demand_c = datetimer(demand_c)
demand_c['date'] = demand_c['datetime'].dt.date
demand_summ = demand_c.groupby(['date'])['demand_hourly'].sum().reset_index()
demand_summ['demand_GWh'] = demand_summ['demand_hourly']/1000000
demand_summ = demand_summ.drop(columns=['demand_hourly'])

# Graphing variables
color_map = {'Water': 'navy', 'Coal': 'dimgrey', 'Gas': 'red'}
tickvals = pd.date_range(start='2010-01-01', end='2023-01-01', freq='YS').astype(str).tolist()
ticktext = list(range(2010, 2023))

In [ ]:
pricefuel = make_subplots(rows=2, cols=1, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.25)

pricefuel = supply_tracer(pricefuel, price_fuel, 'date', 'wa_ask', ['Water', 'Coal', 'Gas'], 'Ask', 1, 1, color_map)

# Bid Price
pricefuel.add_trace(go.Scatter(
    x=price_fuel['date'],
    y=price_fuel['wa_bid'],
    mode='lines',
    name='Bid',
    line=dict(color='slategrey'), 
), row=2, col=1)

pricefuel.update_layout(
    font_color='black', 
    font_family='Latin Modern Math',
    template='plotly_white',
)

pricefuel.update_xaxes(
    dtick='D1',  
    tickformat='%Y-%m-%d', 
    tickvals=tickvals,  
    ticktext=ticktext,  
    row=2, col=1
)

pricefuel.update_xaxes(title_text='Date', row=2, col=1)
pricefuel.update_yaxes(title_text='COP/kWh', row=1, col=1)
pricefuel.update_yaxes(title_text='COP/kWh', row=2, col=1)

pricefuel.show()
pricefuel.write_image('images/dailyprice.pdf', scale=2)

In [ ]:
sdfuel = make_subplots(rows=2, cols=1, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.25)

sdfuel = supply_tracer(sdfuel, supply_fuel_summ, 'date', 'supply_GWh', ['Water', 'Coal', 'Gas'], 'Supply', 1, 1, color_map)

# Demand
sdfuel.add_trace(go.Scatter(
    x=demand_summ['date'],
    y=demand_summ['demand_GWh'],
    mode='lines',
    name='Demand',
    line=dict(color='slategrey'), 
), row=2, col=1)

sdfuel.update_layout(
    font_color='black', 
    font_family='Latin Modern Math',
    template='plotly_white',
)

sdfuel.update_xaxes(
    dtick="D1",  
    tickformat="%Y-%m-%d",  
    tickvals=tickvals,  
    ticktext=ticktext,  
)

sdfuel.update_xaxes(title_text='Date', row=2, col=1)
sdfuel.update_yaxes(title_text='GWh', row=1, col=1)
sdfuel.update_yaxes(title_text='GWh', row=2, col=1)

sdfuel.show()
sdfuel.write_image('images/dailysupplydemand.pdf', scale=2)